# Examples of creation of correspondance tables.

Correspondance tables are created from "raw" data, which comes in a variety of formats, sometimes considerably messy. To arrive to a clean csv file with an agreed format some data cleaning has to be made. Cleaning should be scripted so we are able to reproduce them from raw data, but the steps required to clean the data are very specific to the case in hand. 

# exiobase to isic v4 

this correspondance table does not exist but here we create it from existing ones. 

## Option 1
"raw" data:
- correspondance from exiobase 2 to isic 3 provided by ntnu
- correspondance between isic 3 and 4 provided by UN
- correspondance between exiobase 3 and 2 (non exisisting).

In [1]:
import pandas as pd

from https://unstats.un.org/unsd/classifications/business-trade/tables/ISIC/ISIC31_ISIC4/ISIC31_ISIC4.txt

In [2]:
isic31to4=pd.read_table('../raw/ISIC31_ISIC4.txt',sep=',')
isic31to4.head()

C:\Users\migue\Anaconda3\envs\bw2\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: read_table is deprecated, use read_csv instead.
  """Entry point for launching an IPython kernel.


,ISIC31code,partialISIC31,ISIC4code,partialISIC4,Detail
0,111,1,111,0,NaN
1,111,1,112,0,NaN
2,111,1,113,1,"Growing of sugar beet, roots and tubers"
3,111,1,114,0,NaN
4,111,1,115,0,NaN


dictionary converting isic 3.1 to 4

In [3]:
isic_31_to_4_dict=dict(isic31to4[['ISIC31code','ISIC4code']].values)

Exiobase 2 to Isic 3. From the Exiobase consortium. Provided by Constantin Stadler

In [71]:
exiobase_to_isic3=pd.read_excel('../raw/ISIC REV. 3 - EXIOBASE2.0.xlsx',usecols='B:C')
exiobase_to_isic3.head()

,ISIC REV. 3 - NACE REV. 1,EXIOBASE 2.0
0,Source (unique),NaN
1,1,i01.a;i01.b;i01.c;i01.d;i01.e;i01.f;i01.g;i01....
2,11,i01.a;i01.b;i01.c;i01.d;i01.e;i01.f;i01.g;i01.h
3,111,i01.a;i01.b;i01.c;i01.e;i01.f;i01.g;i01.h
4,112,i01.d;i01.e;i01.h


ok, this needs further processing because codes are given at different levels of aggregation.

In [72]:
exiobase_to_isic3=exiobase_to_isic3.drop(0)

In [73]:
exiobase_to_isic3.columns=['isic3','exiobase2']

convert to string to do regex operations more easily

In [74]:
exiobase_to_isic3.isic3=exiobase_to_isic3.isic3.astype(str)

Identify all the isic codes with 4 digits

In [8]:
exiobase_to_isic3_l4=exiobase_to_isic3[exiobase_to_isic3.isic3.str.match('....')]

In [9]:
exiobase_to_isic3_l4.head(5)

,isic3,exiobase2
23,1010,i10
25,1020,i10
27,1030,i10
29,1110,i11.a;i11.b;i11.c
30,1120,i11.a;i11.b;i11.c


we can iterate the columns and parse the string in exiobase separating by `;`. To create a dictionary of the exiobase codes to isic codes.

In [75]:
ex2_isic3_l4={}
for row in exiobase_to_isic3_l4.iterrows():
    for excode in row[1]['exiobase2'].split(';'):
        excode=excode.strip()
        ex2_isic3_l4[excode]=row[1]['isic3']

In [76]:
len(ex2_isic3_l4)

143

now isic3 to isic 4

In [77]:
ex2_isic4_l4={}
for item in ex2_isic3_l4.items():
    ex2_isic4_l4[item[0]]=str(isic_31_to_4_dict[int(item[1])])

In [78]:
len(ex2_isic4_l4)

143

we only have correspondances between 143 codes in Exiobase 2 to isic v4. Exiobase 3 has 200 activities, thus some are necesarely missing. If only because Exiobase was expanded.

Read existing activity names in Exiobase 3.3.17

In [80]:
ex3=pd.read_excel('../raw/Classifications_v_3_3_17.xlsx',sheet_name='Activities',index_col=[0,1])
ex3.head(3)

Activity code 1 Activity code 2
Contry code Activity name                                                   
AU          Cultivation of paddy rice                  i01.a          A_PARI
            Cultivation of wheat                       i01.b          A_WHEA
            Cultivation of cereal grains nec           i01.c          A_OCER

In [81]:
(ex3.groupby('Contry code').count()==164).all()

Activity code 1    True
Activity code 2    True
dtype: bool

All the countries have the same number of activities, so we can pick one to identify the codes. 

Dictionary of activity names to activity code 1

In [82]:
ex3_descrip_to_actcode1=dict(ex3.loc['AU','Activity code 1'])

Dictionary of activity code 1 to activity names

In [54]:
ex3_actcode1_to_descrip = {v: k for k, v in ex3_descrip_to_actcode1.items()}

Identify the missing ones.

In [31]:
ex3_actcode1_set=set(ex3['Activity code 1'].values)

In [33]:
missing_codes=ex3_actcode1_set.difference(set([k for k in ex2_isic4_l4.keys()]))

In [61]:
for code in missing_codes:
    print(code,'-',ex3_actcode1_to_descrip[code])

i90.5.d - Landfill of waste: Inert/metal/hazardous
i01.f - Cultivation of sugar cane, sugar beet
i24.a - Plastics, basic
i01.a - Cultivation of paddy rice
i90.2.a - Biogasification of food waste, incl. land application
i05 - Fishing, operating of fish hatcheries and fish farms; service activities incidental to fishing (05)
i01.k - Poultry farming
i01.d - Cultivation of vegetables, fruit, nuts
i90.5.c - Landfill of waste: Plastic
i90.2.c - Biogasification of sewage slugde, incl. land application
i24.a.w - Re-processing of secondary plastic into new plastic
i01.o - Wool, silk-worm cocoons
i01.b - Cultivation of wheat
i01.w.1 - Manure treatment (conventional), storage and land application
i90.2.b - Biogasification of paper, incl. land application
i26.a.w - Re-processing of secondary glass into new glass
i01.e - Cultivation of oil seeds
i90.5.f - Landfill of waste: Wood
i01.g - Cultivation of plant-based fibers
i26.d.w - Re-processing of ash into clinker
i01.c - Cultivation of cereal grain

In [57]:
#assume they have not changed.
ex3_isic4_l4=ex2_isic4_l4.copy()

In [70]:
#for codes in missing_codes:
#    print('#',ex3_actcode1_to_descrip[codes])
#    print("ex3_isic4_l4['{}']=".format(codes))

In [ ]:
# Landfill of waste: Inert/metal/hazardous
ex3_isic4_l4['i90.5.d']='5093'
# Cultivation of sugar cane, sugar beet
ex3_isic4_l4['i01.f']='0133'
# Plastics, basic
ex3_isic4_l4['i24.a']=
# Cultivation of paddy rice
ex3_isic4_l4['i01.a']=
# Biogasification of food waste, incl. land application
ex3_isic4_l4['i90.2.a']=
# Fishing, operating of fish hatcheries and fish farms; service activities incidental to fishing (05)
ex3_isic4_l4['i05']=
# Poultry farming
ex3_isic4_l4['i01.k']=
# Cultivation of vegetables, fruit, nuts
ex3_isic4_l4['i01.d']=
# Landfill of waste: Plastic
ex3_isic4_l4['i90.5.c']=
# Biogasification of sewage slugde, incl. land application
ex3_isic4_l4['i90.2.c']=
# Re-processing of secondary plastic into new plastic
ex3_isic4_l4['i24.a.w']=
# Wool, silk-worm cocoons
ex3_isic4_l4['i01.o']=
# Cultivation of wheat
ex3_isic4_l4['i01.b']=
# Manure treatment (conventional), storage and land application
ex3_isic4_l4['i01.w.1']=
# Biogasification of paper, incl. land application
ex3_isic4_l4['i90.2.b']=
# Re-processing of secondary glass into new glass
ex3_isic4_l4['i26.a.w']=
# Cultivation of oil seeds
ex3_isic4_l4['i01.e']=
# Landfill of waste: Wood
ex3_isic4_l4['i90.5.f']=
# Cultivation of plant-based fibers
ex3_isic4_l4['i01.g']=
# Re-processing of ash into clinker
ex3_isic4_l4['i26.d.w']=
# Cultivation of cereal grains nec
ex3_isic4_l4['i01.c']=
# Forestry, logging and related service activities (02)
ex3_isic4_l4['i02']=
# Chemicals nec
ex3_isic4_l4['i24.d']=
# Pigs farming
ex3_isic4_l4['i01.j']=
# P- and other fertiliser
ex3_isic4_l4['i24.c']=
# Manufacture of gas;
ex3_isic4_l4['i40.2.a']=
# Raw milk
ex3_isic4_l4['i01.n']=
# N-fertiliser
ex3_isic4_l4['i24.b']=
# Meat animals nec
ex3_isic4_l4['i01.l']=
# Landfill of waste: Textiles
ex3_isic4_l4['i90.5.e']=
# Animal products nec
ex3_isic4_l4['i01.m']=
# Distribution of gaseous fuels through mains
ex3_isic4_l4['i40.2.b']=
# Manure treatment (biogas), storage and land application
ex3_isic4_l4['i01.w.2']=
# Cattle farming
ex3_isic4_l4['i01.i']=
# Cultivation of crops nec
ex3_isic4_l4['i01.h']=

In [14]:
ex2_isic4_l4_df=pd.DataFrame.from_dict(ex2_isic4_l4,orient='index').reset_index()
ex2_isic4_l4_df.columns=['exiobase2','isic_rev4_l4']
ex2_isic4_l4_df.head(3)

,exiobase2,isic_rev4_l4
0,i10,990
1,i11.a,910
2,i11.b,910


Once "cleaned" it can be exported to csv

In [106]:
ex2_isic4_l4_df.to_csv('../raw/exiobase2_isic_rev4_l4.csv',index=False)

**To be continued ...**

# Entso-e to Exiobase.

load exiobase activity names

In [84]:
ex3=pd.read_excel('../raw/Classifications_v_3_3_17.xlsx',sheet_name='Activities')
ex3.head(3)

,Contry code,Activity name,Activity code 1,Activity code 2
0,AU,Cultivation of paddy rice,i01.a,A_PARI
1,AU,Cultivation of wheat,i01.b,A_WHEA
2,AU,Cultivation of cereal grains nec,i01.c,A_OCER


In [87]:
ex3_activity_names=ex3['Activity name'].unique()

load entso-e activities (provided by Carlos)

In [98]:
entsoe_technologies=pd.read_csv('../raw/bentso_technologies_updated.csv')
entsoe_technologies

,technology
0,Fossil Peat
1,Hydro Water Reservoir
2,Fossil Brown coal/Lignite
3,Nuclear
4,Other
5,Solar
6,Fossil Gas
7,Wind Offshore
8,Biomass
9,Hydro Run-of-river and poundage


one could do some string matching operations to find the equivalences.. e.g. exiobase activities containing electricity in the name.

In [96]:
[s for s in ex3_activity_names if "electricity" in s.lower()] 

['Production of electricity by coal',
 'Production of electricity by gas',
 'Production of electricity by nuclear',
 'Production of electricity by hydro',
 'Production of electricity by wind',
 'Production of electricity by petroleum and other oil derivatives',
 'Production of electricity by biomass and waste',
 'Production of electricity by solar photovoltaic',
 'Production of electricity by solar thermal',
 'Production of electricity by tide, wave, ocean',
 'Production of electricity by Geothermal',
 'Production of electricity nec',
 'Transmission of electricity',
 'Distribution and trade of electricity']

the easiest is to do the matching by hand.

In [102]:
 for t in entsoe_technologies.technology.unique():
        print("'{}':".format(t))

'Fossil Peat':
'Hydro Water Reservoir':
'Fossil Brown coal/Lignite':
'Nuclear':
'Other':
'Solar':
'Fossil Gas':
'Wind Offshore':
'Biomass':
'Hydro Run-of-river and poundage':
'Hydro Pumped Storage':
'Marine':
'Wind Onshore':
'Waste':
'Geothermal':
'Fossil Oil shale':
'Fossil Coal-derived gas':
'Fossil Oil':
'Other renewable':
'Fossil Hard coal':


In [103]:
entsoe_to_exiobase3={
'Fossil Peat':'Production of electricity by coal',
'Hydro Water Reservoir':'Production of electricity by hydro',
'Fossil Brown coal/Lignite':'Production of electricity by coal',
'Nuclear':'Production of electricity by nuclear',
'Other':'Production of electricity nec',
'Solar':'Production of electricity by solar photovoltaic',
'Fossil Gas':'Production of electricity by gas',
'Wind Offshore':'Production of electricity by wind',
'Biomass':'Production of electricity by biomass and waste',
'Hydro Run-of-river and poundage':'Production of electricity by hydro',
'Hydro Pumped Storage':'Production of electricity by hydro',
'Marine':'Production of electricity by tide, wave, ocean',
'Wind Onshore':'Production of electricity by wind',
'Waste':'Production of electricity by biomass and waste',
'Geothermal':'Production of electricity by Geothermal',
'Fossil Oil shale':'Production of electricity by coal',
'Fossil Coal-derived gas':'Production of electricity by coal', #?
'Fossil Oil':'Production of electricity by petroleum and other oil derivatives',
'Other renewable':'Production of electricity by tide, wave, ocean',
'Fossil Hard coal':'Production of electricity by coal',
}

some checks.

In [114]:
#all technologies in entso-e have a exiobase "correspondant"
assert len(set(entsoe_technologies.technology.unique()).difference(set([k for k in entsoe_to_exiobase3.keys()])))==0

In [120]:
entsoe_to_exiobase3_df=pd.DataFrame.from_dict(entsoe_to_exiobase3,orient='index').reset_index()
entsoe_to_exiobase3_df.columns=['entso-e','exiobase3']

In [121]:
entsoe_to_exiobase3_df

,entso-e,exiobase3
0,Fossil Peat,Production of electricity by coal
1,Hydro Water Reservoir,Production of electricity by hydro
2,Fossil Brown coal/Lignite,Production of electricity by coal
3,Nuclear,Production of electricity by nuclear
4,Other,Production of electricity nec
5,Solar,Production of electricity by solar photovoltaic
6,Fossil Gas,Production of electricity by gas
7,Wind Offshore,Production of electricity by wind
8,Biomass,Production of electricity by biomass and waste
9,Hydro Run-of-river and poundage,Production of electricity by hydro


this table can easily be stored into csv for further processing. For example specifying that "fossil hard coal" is a subclass of "Production of electricity by coal". 
The selection of the predicate may not be so easy to automate, but we could have some defaults.